In [1]:
import polars as pl

from mfg_capabilities.config import config

from mfg_capabilities.utils import get_sqlalchemy_engine

CATALOG = "manufacturing_dev"
SCHEMA = "work_agent_barney"


pl.Config.set_tbl_rows(20)  # to set the number of rows displayed)
# pl.Config.set_tbl_cols(10) # to set the number of columns
# pl.Config.set_fmt_str_lengths(50) # to set the max string length displayed

polars.config.Config

## Load line and product data

In [2]:
# I manually cleaned the Excel sheets from central planning
# and stored in a single excel file
PRODUCTS_PATH = config.data_dir / "from_central_planning" / "products_from_capacity.xlsx"

# Read all columns as strings first to handle concatenated CSVs with extra headers
products_raw = pl.read_excel(
    PRODUCTS_PATH,
    sheet_name="Sheet1",
    has_header=True,
    infer_schema_length=0  # Treat all columns as strings
)

display(products_raw)

Year,Quarter,Plant Number,Plant Name,Line Name,Product
str,str,str,str,str,str
"""2025""","""1""","""102""","""Country Oven Bakery""","""Bread Line""","""B19011 ITALIAN BREAD"""
"""2025""","""1""","""102""","""Country Oven Bakery""","""Bread Line""","""B19017 COUNTRY FRENCH"""
"""2025""","""1""","""102""","""Country Oven Bakery""","""Bread Line""","""B19019 SEEDED RYE BREAD"""
"""2025""","""1""","""102""","""Country Oven Bakery""","""Bread Line""","""B19068 PUMPERNICKEL BREAD"""
"""2025""","""1""","""102""","""Country Oven Bakery""","""Bread Line""","""B19100 FRENCH STIX"""
"""2025""","""1""","""102""","""Country Oven Bakery""","""Bread Line""","""B19180 SWEET ITALIAN STIX"""
"""2025""","""1""","""102""","""Country Oven Bakery""","""Bread Line""","""B19183 WHITE MOUNTAIN BREAD"""
"""2025""","""1""","""102""","""Country Oven Bakery""","""Bread Line""","""B19184 Wheat Mountain"""
"""2025""","""1""","""102""","""Country Oven Bakery""","""Bread Line""","""B19185 ROUND PUMPERNICKLE"""


In [3]:
products_cleaned = (
    products_raw
    .filter(~pl.col("Product").str.contains("(?i)none"))
    .select("Plant Number", "Plant Name", "Line Name", "Product")
    .sort("Plant Number", "Plant Name", "Line Name", "Product")
)

display(products_cleaned)

Plant Number,Plant Name,Line Name,Product
str,str,str,str
"""049""","""State Avenue Foods""","""670 SPICE""","""K00106 HT ONION POWDER 4OZ."""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48302 PS POPPY SEED"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48302 PS POPPY SEED"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48307 PS GROUND CLOVES"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48309 PS WHOLE CLOVES"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48309 PS WHOLE CLOVES"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48463 PS GROUND CUMIN"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48463 PS GROUND CUMIN"""
"""049""","""State Avenue Foods""","""670 SPICE""","""K48464 PS THYME LEAVES"""


## Load data work_agent_barney

In [5]:
engine = get_sqlalchemy_engine(
    catalog=CATALOG,
    schema=SCHEMA
)

query = "SELECT * FROM plm_spec_cross_references_silver"
plm_spec_cross_refs = pl.read_database(query, engine)

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
